In [ ]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b3.tar.gz

!tar -xf /kaggle/working/noisy_student_efficientnet-b3.tar.gz

In [ ]:
!tar -xf /kaggle/working/noisy_student_efficientnet-b3.tar.gz

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/python/keras/applications/efficientnet_weight_update_util.py

In [ ]:
!python ./efficientnet_weight_update_util.py --model b3 --notop --ckpt ./noisy_student_efficientnet-b3/model.ckpt --o ./efficientnetb3_notop.h5

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB3
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
import os, cv2, json

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

In [ ]:
# For easy acces to files
WORK_DIR = "../input/cassava-leaf-disease-classification/"
os.listdir(WORK_DIR)

In [ ]:
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as file:
    labels = json.load(file)
    
labels

In [ ]:
data = pd.read_csv(WORK_DIR + "train.csv")

In [ ]:
#change for the ImageDatagen and flow_from_dataframe
data.label = data.label.astype("str")

In [ ]:
data.dtypes

In [ ]:
data.shape[0]

In [ ]:
data.label.value_counts()

In [ ]:
IMG_SIZE = 300

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data.sample(9).reset_index(drop=True)

for i in range(8):
    plt.subplot(2,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

In [ ]:


train_generator = ImageDataGenerator(
                                    #featurewise_center=False,                                    
                                    #samplewise_center=False,
                                    #featurewise_std_normalization=False,
                                    #samplewise_std_normalization=False, 
                                    #zca_whitening=False,
                                    #zca_epsilon=1e-06,
                                    rotation_range=270,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.1,0.9],
                                    shear_range=25,
                                    zoom_range=0.3,
                                    channel_shift_range=0.2,
                                    #fill_mode="nearest",
                                    #cval=0.0,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    #rescale=None,
                                    #preprocessing_function=None,
                                    #data_format=None,
                                    validation_split=0.2,
                                    #dtype=None,
) \
        .flow_from_dataframe(
                            data,
                            directory = WORK_DIR + "train_images",
                            x_col = "image_id",
                            y_col = "label",
                            #weight_col = None,
                            target_size = (IMG_SIZE, IMG_SIZE),
                            #color_mode = "rgb",
                            #classes = None,
                            class_mode = "categorical",
                            batch_size =32,
                            shuffle = True,
                            #seed = 34,
                            #save_to_dir = None,
                            #save_prefix = "",
                            #save_format = "png",
                            subset = "training",
                            #interpolation = "nearest",
                            #validate_filenames = True
)


In [ ]:
valid_generator = ImageDataGenerator(
                                    validation_split = 0.2
) \
        .flow_from_dataframe(
                            data,
                            directory = WORK_DIR + "train_images",
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = 32,
                            shuffle = True,
                            #seed = 34,
                            subset = "validation")

In [ ]:
valid_generator.class_indices

In [ ]:
def modelEfficientNetB3():
    
    model = models.Sequential()
    
    #model.add(EfficientNetB3(include_top = False, weights = "./efficientnetb3_notop.h5",input_shape=(IMG_SIZE,IMG_SIZE, 3)))
    
    pre_trained_model = EfficientNetB3(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, weights = "./efficientnetb3_notop.h5",drop_connect_rate=0.2)
    pre_trained_model.trainable = True
     # We unfreeze layers while leaving BatchNorm layers frozen
    for layer in pre_trained_model.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
            
    model.add(pre_trained_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(5, activation = "softmax"))
    
    return model 

In [ ]:
model = modelEfficientNetB3()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import utils

utils.plot_model(model)

In [ ]:
model_check = ModelCheckpoint(
                            "./firstTry.h5",
                            monitor = "val_loss",
                            verbose = 1,
                            save_best_only = True,
                            save_weights_only = False,
                            mode = "min")

In [ ]:
early_stop= EarlyStopping(
                                monitor = "val_loss",
                                min_delta=0.001,
                                patience=7,
                                verbose=1,
                                mode="min",
                                #baseline=None,
                                restore_best_weights=False)

In [ ]:
reduce_lr = ReduceLROnPlateau(
                                monitor="val_loss",
                                factor=0.1,
                                patience=2,
                                verbose=1,
                                mode="min",
                                min_delta=0.0001,
                                #cooldown=0,
                                #min_lr=0
)

In [ ]:
model.compile(optimizer =  Adam(learning_rate = 1e-4),
            loss = CategoricalCrossentropy(label_smoothing=0.3,reduction="auto",name="categorical_crossentropy"),
            metrics = ["accuracy"])

callbacks = [model_check,reduce_lr]

In [ ]:
history = model.fit_generator(train_generator,
                            epochs = 30,
                            validation_data = valid_generator,
                            callbacks = [model_check,early_stop,reduce_lr])


In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'], 'b*-', label="train_acc")
plt.plot(history.history['val_accuracy'], 'r*-', label="val_acc")
plt.grid()
plt.title("train_acc vs val_acc")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'], 'b*-', label="train_loss")
plt.plot(history.history['val_loss'], 'r*-', label="val_loss")
plt.grid()
plt.title("train_loss - val_loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()